# Notebook showing a fast and efficient way to load Sentinel-2 L1C data into xarray

In [7]:
import glob
import os
import numpy as np

import pandas as pd
import geopandas as gpd
import xarray as xr

import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy.crs as ccrs
from matplotlib.colors import ListedColormap
from s2driver import driver_S2_SAFE as S2

import json
import requests
from datetime import datetime
from multiprocessing import Pool
import subprocess

from eoreader.keywords import CLEAN_OPTICAL
import GRSdriver

opj = os.path.join

GRSdriver.__version__

'1.0.5'

### Download L1C images through S3 protocol, please visit https://documentation.dataspace.copernicus.eu/APIs/S3.html

In [12]:
odir = '/data/satellite/Sentinel-2/L1C/'
date_start='2021-01-01T00:00:00.000Z' 
date_end ='2021-12-31T00:00:00.000Z'
tiles=['17MQT']#'30PYT']
date_start='2018-01-01T00:00:00.000Z' 
date_end ='2018-12-31T00:00:00.000Z'
tiles=['31TFJ']
cloud_cover_max = 60      


cmd = []
for tile in tiles:
    json = requests.get("https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name eq 'SENTINEL-2' "+
                    " and Attributes/OData.CSC.DoubleAttribute/any(att:att/Name eq 'cloudCover' and att/OData.CSC.DoubleAttribute/Value lt {:.2f})".format(cloud_cover_max)+
                    " and Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'productType' and att/OData.CSC.StringAttribute/Value eq 'S2MSI1C')"+
                    " and contains(Name,'"+tile+"') "+
                    # next line is to keep latest baseline only (comment to get baseline < 4)
                    " and (contains(Name,'_N04') or contains(Name,'_N05')) "+
                    " and ContentDate/Start gt "+date_start+" and ContentDate/Start lt "+date_end+"&$orderby=ContentDate/Start&$top=1000").json()
    df = pd.DataFrame.from_dict(json['value'])
    
    
    
    for irow,info in df.iterrows():
        #print(info['Name'])
        date=datetime.strptime(info['Name'].split('_')[2],'%Y%m%dT%H%M%S')
        date.year
        odir_ = opj(odir,tile,str(date.year),str(date.month).zfill(2),str(date.day).zfill(2))
        cmd.append('s3cmd -c ~/.s3cfg sync s3:/'+info['S3Path']+' '+odir_)
cmd

['s3cmd -c ~/.s3cfg sync s3://eodata/Sentinel-2/MSI/L1C/2018/12/01/S2A_MSIL1C_20181201T103401_N0207_R108_T31TFJ_20181201T110543.SAFE /data/satellite/Sentinel-2/L1C/31TFJ/2018/12/01',
 's3cmd -c ~/.s3cfg sync s3://eodata/Sentinel-2/MSI/L1C/2018/12/04/S2A_MSIL1C_20181204T104411_N0207_R008_T31TFJ_20181204T111726.SAFE /data/satellite/Sentinel-2/L1C/31TFJ/2018/12/04']

In [13]:

def call(command):
    print(command)
    pipeline_out = subprocess.call(command, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT, shell=True)
    return

ncore=4
with Pool(processes=ncore) as pool:
    pool.map(call, cmd)
    pool.close

s3cmd -c ~/.s3cfg sync s3://eodata/Sentinel-2/MSI/L1C/2018/12/04/S2A_MSIL1C_20181204T104411_N0207_R008_T31TFJ_20181204T111726.SAFE /data/satellite/Sentinel-2/L1C/31TFJ/2018/12/04s3cmd -c ~/.s3cfg sync s3://eodata/Sentinel-2/MSI/L1C/2018/12/01/S2A_MSIL1C_20181201T103401_N0207_R108_T31TFJ_20181201T110543.SAFE /data/satellite/Sentinel-2/L1C/31TFJ/2018/12/01



In [16]:
import requests

url = f"https://zipper.dataspace.copernicus.eu/odata/v1/Products("+df.Id[0]+")/$value"

headers = {"Authorization": f"Bearer {access_token}"}

session = requests.Session()
session.headers.update(headers)
response = session.get(url, headers=headers, stream=True)

with open("product.zip", "wb") as file:
    for chunk in response.iter_content(chunk_size=8192):
        if chunk:
            file.write(chunk)